In [ ]:
# Get the template
from app.src.helpers.template_getter import get_template
contract = get_template('sample_t')

#print(contract.to_sym())

contract.print_all_strings()

In [ ]:
# Select the parameter
from app.templates.sample.t.nl_template import parameters

pki_dict = {i+1: pk for i,pk in enumerate(parameters)}

for pki in pki_dict:
    print(f'{pki} - {pki_dict[pki]}')

selected_pki = input('Select a parameter: ')
selected_pk = pki_dict[int(selected_pki)]
parm_spec = parameters[selected_pk]

print(f'\nParameter: {selected_pk}')

In [ ]:
from app.src.grammar.grammar_generator import GrammarGenerator
from app.src.grammar.grammar_config import GrammarGeneratorConfig
from app.src.grammar.grammar_selector import GrammarSelector
from app.src.grammar.manual_node_selector import ManualGrammarNodeSelector
from app.src.grammar.helpers.domain_timepoint_extractor import DomainTimepointExtractor

domain_timepoint_extractor = DomainTimepointExtractor()
gg = GrammarGenerator(domain_timepoint_extractor)
grammar_config = GrammarGeneratorConfig(parm_spec.op_codes)
grammar_root = gg.generate(contract, grammar_config)

inner_selector = ManualGrammarNodeSelector()
gs = GrammarSelector(inner_selector)
selection = gs.select(grammar_root)


## Use Runner

In [ ]:
from app.src.runner import Runner

# Run for all parameters
for parm_config in parm_spec.configs:
    runner = Runner(contract, parm_config)
    contract = runner.update_contract(selection)

print(contract.to_sym())


## Step through

In [ ]:
from app.src.operations.helpers.default_event_getter import DefaultEventGetter

# Extract the default event
deg = DefaultEventGetter()

# Handle the case where there is none...
default_event = deg.get(contract.contract_spec, parm_config)

if default_event:
    print(f'Default Event: {default_event.to_sym()}')
else:
    print('No default event')

In [ ]:
#default_event = None

In [ ]:
from app.src.frames.frame_checker_constuctor import FrameCheckerConstructor

frame_checker = FrameCheckerConstructor.construct()
frames = frame_checker.check_all_frames(selection.nodes)

for x in frames:
    print(type(x), x.to_text())

main_frame = frames[0]
op_code = main_frame.op_code

In [ ]:
# Create the basket
from app.classes.grammar.selected_node import Basket
basket = Basket()
basket.default_event = default_event
basket.initial_norm = contract.contract_spec.__dict__[parm_config.norm_type][parm_config.norm_id]

In [ ]:
# Generate the Symboleo
symboleo_obj = selection.to_obj(basket)
print(symboleo_obj.to_sym())

In [ ]:
# Updating
from app.src.operations.contract_updater_builder import ContractUpdaterBuilder

contract_updater = ContractUpdaterBuilder.build()

updated_contract = contract_updater.update(op_code, contract, symboleo_obj, parm_config)


In [ ]:
print(updated_contract.to_sym())